A simple script to determine (Either from an excel, or python dictionary) how much money is owed to the guy who covered the bill.  Fairly accounting for tip and tax relative to how much your items cost.

If you replicate, be careful to ensure you don't have rounding and floating point issues that would inflate the money owed.

In [ ]:
import pandas as pd
#import will2live
import copy


In [2]:
#GIVEN VALUEs

items = [
    ("Pizza", 20, ["Alice", "Bob"]),
    ("Pasta", 15, ["Alice"]),
    ("Salad", 10, ["Bob", "Charlie"]),
    ("Drinks", 5, ["Charlie"])
]


tax_amount=10.97
tip_amount=10.73
total_tax_tip=tax_amount+tip_amount

In [17]:
def money_owed(items,tax_amount,tip_amount):
    person_list=[]

    for item,cost,names in items:
        for name in names:
            person_list.append(name)

    person_list= list(set(person_list))
    person_dict=dict.fromkeys(person_list)

    for x in person_dict.keys():
        person_dict[x]=0

    person_dict_percent_of_bill=copy.deepcopy(person_dict)
    person_dict_final=copy.deepcopy(person_dict)

    running_total_preTaxTip=0

    for item,cost,names in items:
        # determine the cost per person when food is split
        split_cost_of_item=cost/len(names)
        #print(split_cost_of_item)
        # add that to the total bill cost
        running_total_preTaxTip+=split_cost_of_item*len(names)

        for name in names:
            person_dict[name]+=split_cost_of_item
    total_with_additions=running_total_preTaxTip+total_tax_tip

    for person in person_dict_percent_of_bill.keys():
        #print(person)
    
    #person_dict_percent_of_bill[person]=person_dict[person]/total_with_additions
        person_dict_percent_of_bill[person]=person_dict[person]/running_total_preTaxTip
    
    
    for person in person_dict:
        #person_tax_tip_to_pay=person_dict_percent_of_bill[person]*total_tax_tip
        #print(person_tax_tip_to_pay)
        person_tip_to_pay=person_dict_percent_of_bill[person]*tip_amount
        person_tax_to_pay=person_dict_percent_of_bill[person]*tax_amount
        #print(person_tax_tip_to_pay,sum([person_tax_to_pay,person_tip_to_pay]))
        person_tax_tip_to_pay=person_tax_to_pay+person_tip_to_pay
        person_final_total=person_tax_tip_to_pay+person_dict[person]
        person_dict_final[person]=person_final_total

        
    for person in person_dict_final.keys():
        #print(person)
        value=person_dict_final[person]
        value=round(value,2)
        person_dict_final[person]=value

    return(person_dict_final)

In [18]:
def owed_from_xl(filepath,tax_amount,tip_amount):
    df=pd.read_excel(filepath)
    items = []
    for index, row in df.iterrows():
        item_name = row['Item']
        #print(item_name)
        item_cost = row['amount']
        #print(item_cost)
        # Collect all non-null values from the person columns
        consumers=[]
        ##consumers = [row[col] for col in df.columns[2:] if not pd.isnull(row[col])]
        for col in df.columns[2:]:
            if str(row[col])!='nan':
                consumers.append(row[col])
                #print(row[col])

        items.append((item_name,item_cost,consumers))
    
    return(money_owed(items,tax_amount,tip_amount))

In [19]:
filepath = 'items_chineserestaurant1.xlsx'  

owed_from_xl(filepath,10,10)

{'ariel': 13.66,
 'linda': 16.39,
 'haoran': 20.23,
 'scott': 28.89,
 'nethra': 17.58,
 'lea/ds': 23.23,
 'ds': 25.37,
 'brittany': 28.2}

In [21]:
pd.read_excel(filepath)

,Item,amount,names,Unnamed: 3,Unnamed: 4
0,green bean jelly,8.95,haoran,NaN,NaN
1,slice bf maw sze,8.95,haoran,NaN,NaN
2,chicken fried rice 1.5,21.45,scott,nethra,NaN
3,ck fried rice,14.50,ariel,lea/ds,NaN
4,bbq combo,24.95,brittany,NaN,NaN
5,turnip cake,6.35,lea/ds,NaN,NaN
6,crepe shrimp,6.95,lea/ds,NaN,NaN
7,beef lo mein,22.45,ds,NaN,NaN
8,fried bf rice noodle,14.50,linda,NaN,NaN
9,veg roll,14.50,scott,nethra,ariel
